# CatBoost in Python (Colab)

This Colab notebook is a practical, **end-to-end** template:
- Load a tabular dataset
- Train a baseline model
- Evaluate
- Tune hyperparameters (RandomizedSearchCV)
- Interpret with feature importance + SHAP


## 1) Install & Imports
Uncomment installs if needed in Colab.


In [ ]:
# If needed:
# !pip -q install catboost shap
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import matplotlib.pyplot as plt

RANDOM_STATE = 42


## 2) Dataset (Breast Cancer)
We use a built-in dataset so the notebook runs anywhere.


In [ ]:
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
X_train.shape, X_test.shape


## 3) Baseline CatBoost


In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(
    iterations=800,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=RANDOM_STATE,
    verbose=100,
)
cat.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

proba = cat.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)
print('Accuracy:', accuracy_score(y_test, pred))
print('ROC AUC:', roc_auc_score(y_test, proba))
print('\n', classification_report(y_test, pred))


## 4) Hyperparameter tuning (RandomizedSearchCV)


In [ ]:
param_dist = {
    'iterations': [300, 600, 1000],
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.03, 0.07],
    'l2_leaf_reg': [1, 3, 5, 10],
    'bagging_temperature': [0.0, 0.5, 1.0],
}

search = RandomizedSearchCV(
    estimator=CatBoostClassifier(
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=RANDOM_STATE,
        verbose=0,
    ),
    param_distributions=param_dist,
    n_iter=20,
    scoring='roc_auc',
    cv=5,
    verbose=1,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)
search.fit(X_train, y_train)
print('Best params:', search.best_params_)
print('Best CV ROC AUC:', search.best_score_)

best_cat = search.best_estimator_
proba = best_cat.predict_proba(X_test)[:, 1]
print('Test ROC AUC:', roc_auc_score(y_test, proba))


## 5) Feature importance + SHAP


In [ ]:
import pandas as pd
importances = pd.Series(best_cat.get_feature_importance(), index=X.columns).sort_values(ascending=False)
display(importances.head(15))

importances.head(15).iloc[::-1].plot(kind='barh')
plt.title('Top 15 Feature Importances (CatBoost)')
plt.show()


In [ ]:
import shap
shap.initjs()

sample = X_test.sample(n=min(200, len(X_test)), random_state=RANDOM_STATE)
shap_values = best_cat.get_feature_importance(data=sample, type='ShapValues')
shap_vals = shap_values[:, :-1]  # last column is expected value

shap.summary_plot(shap_vals, sample, plot_type='bar')
shap.summary_plot(shap_vals, sample)
